# Function to set up text alert messages

This is a scratch notebook to develop a script for messaging people about a spike at a sensor near them

## Import Packages

In [1]:
### Import Packages

# File manipulation

import os # For working with Operating System
import requests # Accessing the Web
import datetime as dt # Working with dates/times

# Database 

# import psycopg2
# from psycopg2 import sql

# Analysis

import numpy as np
import geopandas as gpd
import pandas as pd

# Get CWD

cwd = os.getcwd()

In [2]:
# Load some data
# Eventually this will be a database thing

datapath = os.path.join(cwd, '..', '..', 'Data')

sensor_info = gpd.read_file(os.path.join(datapath, 'PurpleAir_Stations.geojson'))

intersections = gpd.read_file(os.path.join(datapath, 'Road_Intersections.geojson'))

## New Alert

In [6]:
# Dummy variables

sensor_index = 168327
reading = 32.7
intersection_index = 3


In [42]:
# Get direction and distance from intersection they signed up for
# Using PostGIS

f'''
WITH sensor_data as
(
SELECT * 
FROM PurpleAir Stations p
WHERE p.sensor_index = {sensor_index}
),
intersection_data as
(
SELECT *
FROM Road Intersections i
WHERE i.intersection_index = {intersection_index}
)
SELECT degrees(ST_Azimuth(i.geom, p.geom)), -- https://gis.stackexchange.com/questions/54427/how-to-find-out-direction-postgis
ST_Distance(i.geom, p.geom),
i.ns_cross_street,
i.ew_cross_street
FROM sensor_data p, intersection_data i
'''

# Will return some values that we will unpack...

degree = 340
distance = 300
ns_cross_street = 'Hennepin'
ew_cross_street = 'Lake'

In [60]:
def degrees_to_cardinal(degree):
    '''
    note: this is highly approximate...
    from https://gist.github.com/RobertSudwarts/acf8df23a16afdb5837f
    '''
    dirs = np.array(["N", "NNE", "NE", "ENE", "E", "ESE", "SE", "SSE",
                     "S", "SSW", "SW", "WSW", "W", "WNW", "NW", "NNW"], dtype='U')
    ix = int(np.round(degree / (360. / len(dirs)))) % 16
    return dirs[ix]

In [61]:
direction_string = degrees_to_cardinal(degree)

In [64]:
message = f'''SPIKE ALERT! Sensor {sensor_index} is reading at {reading} micrograms/meter^3.
This sensor is about {distance} meters {direction_string} from the intersection of {ew_cross_street} and {ns_cross_street}.
You are receiving this text because you signed up for SpikeAlerts. 
Please reply with STOP to be removed from this list.'''

In [65]:
print(message)

SPIKE ALERT! Sensor 168327 is reading at 32.7 micrograms/meter^3.
This sensor is about 300 meters NNW from the intersection of Lake and Hennepin.
You are receiving this text because you signed up for SpikeAlerts. 
Please reply with STOP to be removed from this list.


## End Alert

In [55]:
# Dummy Variables

sensor_index = 168327
duration = 60
max_reading = 50

In [56]:
message = f'''Spike alert for sensor {sensor_index} is over.
This event lasted {duration} minutes with a maximum value of {max_reading} micrograms/meter^3.
You are receiving this text because you signed up for SpikeAlerts. 
Please reply with STOP to be removed from this list.'''

In [57]:
print(message)

Spike alert for sensor 168327 is over.
This event lasted 60 minutes with a maximum value of 50 micrograms/meter^3.
You are receiving this text because you signed up for SpikeAlerts. 
Please reply with STOP to be removed from this list.
